# Import libraries 

In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from scipy.optimize import minimize, fmin_slsqp
from datetime import date, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pylab

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import cross_val_predict

In [3]:
from sklearn.model_selection import StratifiedKFold, KFold

In [4]:
import lightgbm as lgb 

# Import data 

In [5]:
%%time
# train and sub format already merged with weather and holidays (plus the day before holiday)
train = pd.read_csv('train.csv')

CPU times: user 337 ms, sys: 36.8 ms, total: 374 ms
Wall time: 370 ms


In [6]:
train.columns = ['Timestamp','ForecastId','Value']

In [7]:
def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    X['hour'] = X['Timestamp'].dt.hour
    X['minute'] = X['Timestamp'].dt.minute
    X['minute'] = X['minute'] // 15 * 15
    
    return X

In [8]:
train = time_preprocess(train)

In [9]:
train.head()

,Timestamp,ForecastId,Value,year,month,day,week_day,hour,minute
0,2015-01-01,0,91600,2015,1,1,3,0,0
1,2015-01-02,0,136500,2015,1,2,4,0,0
2,2015-01-03,0,335400,2015,1,3,5,0,0
3,2015-01-04,0,379000,2015,1,4,6,0,0
4,2015-01-05,0,344100,2015,1,5,0,0,0


## validate 

In [10]:
def MeanEncodingTransforming(X, y, X_test, how_to_fill):
    
    # mean encoding for lgb
    
    X_train = pd.concat([X, y], axis=1)
    mean_values = X_train.groupby(X_train.columns[0]).agg(how_to_fill).to_dict()['Value']
    X_train = X_train.drop(y.columns[0], axis=1)
    X_train = X_train.replace(mean_values)
    X_test = X_test.replace(mean_values)
    
    return X_train, X_test

def feature_preprocessing(train, test, cat_cols, cat_type='ohe'):
    
    # ohe or mean encoding preprocessing for the lgb 
    
    X_train, X_test = train[cat_cols].copy(), test[cat_cols].copy()

    if (cat_type=='mean_enc'):
        for j in ['mean', 'max', 'min', 'median']:
            for i in cat_cols:
                X_train[i], X_test[i] = MeanEncodingTransforming(X_train[i], train[['Value']], X_test[i], j)
                X_train[i].columns = [i+'_'+j]
                X_test[i].columns = [i+'_'+j]
                
    if (cat_type=='ohe'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        X_train = ohe.fit_transform(train[cat_cols])
        X_test = ohe.transform(test[cat_cols])
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
    
    return X_train, X_test 

In [11]:
def train_groupby(train, test, window, how):
    
    # simple groupby prediction 
    
    time_delta = list((test['Timestamp'].iloc[-1:]  - train['Timestamp'].iloc[1] ).dt.days)[0]
    
    if (time_delta < 21):
        mean_values = train[['Value', 'hour', 'minute']][-window:].groupby(['hour', 'minute']).agg(how).reset_index()
        mean_values.columns = ['hour','minute', 'pred']
        test = pd.merge(test, mean_values, how='left', on = ['hour','minute'])
        
    if ((time_delta >=21) & (time_delta < 365)):
        mean_values = train[['Value', 'week_day', 'hour', 'minute']][-window:].groupby(['week_day', 'hour', 'minute']).agg(how).reset_index()
        mean_values.columns = ['week_day','hour','minute', 'pred']
        test = pd.merge(test, mean_values, how='left', on = ['week_day','hour','minute'])  
        
    if ( time_delta >=365 ):
        mean_values = train[['Value', 'month', 'week_day', 'hour', 'minute']][-window:].groupby(['month', 'week_day', 'hour', 'minute']).agg(how).reset_index()
        mean_values.columns = ['month', 'week_day','hour','minute', 'pred']
        test = pd.merge(test, mean_values, how='left', on = ['month', 'week_day','hour','minute'])  

    if (np.sum(test['pred'].isnull()) / len(test) > 0.3):
        test = test.drop('pred',axis=1)
        mean_values = train[['Value', 'hour', 'minute']][-window:].groupby(['hour', 'minute']).agg(how).reset_index()
        mean_values.columns = ['hour','minute', 'pred']
        test = pd.merge(test, mean_values, how='left', on = ['hour','minute'])
    
    return test['pred'].fillna(np.mean(train['Value']))

In [12]:
def train_mean(train, window):
    
    # return mean value from train for the window 
    
    mean_value = np.mean(train['Value'].reset_index(drop=True)[-window:])
   
    return mean_value


In [13]:
def validate_lgb(X_train, y_train, X_valid, y_valid):
    
    
    d1 = lgb.Dataset(X_train, y_train, weight=np.linspace(0.5, 1, X_train.shape[0]))
    d2 = lgb.Dataset(X_valid, y_valid)
    
    params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.160042,
        'random_state':42
    }
    
    gbm = lgb.train(params, d1, verbose_eval=None, valid_sets=d2, 
                    num_boost_round=50000, early_stopping_rounds=100)
    
    y_hat = gbm.predict(X_valid)
    opt_boost_rounds = gbm.best_iteration
    
    return y_hat, opt_boost_rounds 



def train_lgb(X_train, y_train, X_test, opt_boost_rounds):
    
    d1 = lgb.Dataset(X_train, y_train, weight=np.linspace(0.5, 1, X_train.shape[0]))
    
    params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.160042,
        'random_state':42
    }
    
    gbm = lgb.train(params, d1, verbose_eval=None, num_boost_round=opt_boost_rounds)
    
    y_hat = gbm.predict(X_test)

    return y_hat

In [14]:
def train_rf(X_train, y_train, X_valid):

    rf = RandomForestRegressor(max_features='sqrt', n_estimators=142, n_jobs=-1, random_state=4224)
    rf.fit(X_train, y_train, sample_weight=np.linspace(0.5, 1, X_train.shape[0]) )
    y_hat = rf.predict(X_valid)
    
    return y_hat

In [15]:
def calc_score(pred, fact, index_mult):
    return mean_absolute_error(pred,fact)

In [16]:
def combine_y_hats(y_hats):
    X_stack = pd.DataFrame({})
    for i in range(0, len(y_hats)):
        X_stack['stack'+str(i)] = y_hats[i]
    return X_stack

In [17]:
def train_stack(X_stack, y, model):
    model.fit(X_stack, y)
    return model

In [18]:
def make_harmonic_features(x, col, period=24):
    x['sin_'+col] = np.sin(x[col] * 2 * np.pi / period)
    x['cos_'+col] = np.cos(x[col] * 2 * np.pi / period)
    x = x.drop(col, axis=1)
    return x

## predict

In [19]:
train['minutes_in_day'] = train['hour']*60 + train['minute']

In [20]:
train['holidays'] = 0

In [21]:
train.head()

,Timestamp,ForecastId,Value,year,month,day,week_day,hour,minute,minutes_in_day,holidays
0,2015-01-01,0,91600,2015,1,1,3,0,0,0,0
1,2015-01-02,0,136500,2015,1,2,4,0,0,0,0
2,2015-01-03,0,335400,2015,1,3,5,0,0,0,0
3,2015-01-04,0,379000,2015,1,4,6,0,0,0,0
4,2015-01-05,0,344100,2015,1,5,0,0,0,0,0


def lin_reg(x):
    return np.sum( (X_valid_v['Value'] - np.dot(X_valid_stack,x) )**2 ) 


def constr_sum(x):
    return np.sum(x)-1

result = minimize(lin_reg, [0,0,0,0], method='SLSQP',
                   bounds=((0.0,1.0), (0.0,1.0), (0.0,1.0), (0.0,1.0)), 
                   constraints=({'type': 'eq', 'fun': lambda x: np.sum(x)-1}), 
                   tol=None, callback=None, options={'maxiter':100, 'disp':True})
x = result['x']
y_lr_minimize = np.sum(X_valid_stack*x, axis=1)
print(result)


import numpy as np
from scipy.optimize import minimize, fmin_slsqp

def lin_reg(x):
    # featurex - matrix of X 
    # x == b in linear regression 
    return np.sum( (y_true - np.dot(features,x) )**2 ) 

def constr_sum(x):
    return np.sum(x)-1

x0 = [0,0,0,0]

result = minimize(lin_reg, x0 , method='SLSQP',
                   bounds=((0.0,1.0), (0.0,1.0), (0.0,1.0), (0.0,1.0)), 
                   constraints=({'type': 'eq', 'fun': lambda x: np.sum(x)-1}), 
                   tol=None, callback=None, options={'maxiter':100, 'disp':True})
print(result)

In [22]:
def nan_fill(X):
    
    values_dict = dict(X[['Timestamp', 'Value']].values)
    
    for i in np.where(X['Value'].isnull())[0]:
        val = []
        for j in [-365, -14, -7, 7, 14, 365]:
            ind = X['Timestamp']==(X['Timestamp'][i]+timedelta(days=j))
            value = list(X['Value'][ind])
            if (value!=[]):
                val.append(value[0])
        
        X['Value'][i] = np.median(val)
        # X['Value'] = X['Value'].interpolate()
    
    return X

In [23]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

def validate_stack(X_stack, y, model):
    y_cros_val_pred = cross_val_predict(model, X_stack, y=y, cv=5, n_jobs=-1)
    return y_cros_val_pred

def cv_lr(X_stack, y, model):
    
    # cros val lasso on X_stack 
    
    y_cros_val_pred = pd.DataFrame({})
    coefs = []
    
    for train_index, test_index in kf.split(X_stack):
        X_train, X_test = X_stack.iloc[train_index], X_stack.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model_temp = model 
        model_temp.fit(X_train,y_train)
        y_hat = model_temp.predict(X_test)
        coefs.append(model_temp.coef_)
        
        temp_df = pd.DataFrame({'id': test_index, 'Value': y_hat})
        y_cros_val_pred = pd.concat([y_cros_val_pred, temp_df], axis=0)
      
    y_cros_val_pred = y_cros_val_pred.sort_values(by='id')
    
    #divide by sum - due to overfit of model, with dividing - coefs ~ weights of models
    return list(y_cros_val_pred['Value']) / np.sum(np.mean(coefs,axis=0)) 

In [24]:
def w_avg(x):       
    
    if (len(x)==1):
        a = x
    
    if (len(x)==2):
        a = (np.array([0.25, 0.75])*x).mean()
        
    if (len(x)==3):
        a = (np.array([0.15, 0.25, 0.6])*x).mean()
        
    if (len(x)==4):
        a = (np.array([0.1, 0.2, 0.3, 0.4])*x).mean()
    
    return a

In [26]:
%%time

losses = []
iids = []

sub = pd.DataFrame({})
cat_cols = ['week_day','hour','minute','month', 'holidays']

for i in pd.unique(train['ForecastId']):
 
    rf_stack = RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=-1)
    lr_stack = Lasso(alpha=1, fit_intercept=False, max_iter=3000, tol=0.0001, positive=True, random_state=424142)

    # prepare train and test Dfs 
    days_in_train = 90
    X_train = train[train['ForecastId']==i].reset_index(drop=True)[-days_in_train:].reset_index(drop=True)
    
    # train on last 1.5 year 
    na_values_before = np.sum(X_train['Value'].isnull())
    X_train = nan_fill(X_train)
    na_values_after = np.sum(X_train['Value'].isnull())
    X_train = X_train.dropna(subset=['Value']).reset_index(drop=True)
    
    X_train['Value_log'] = np.log1p(X_train['Value'])
    
    # drop outliers in train data set 
    #up_border = X_train['Value'].quantile(0.985)
    #low_border = X_train['Value'].quantile(0.015)
    #X_train = X_train[(X_train['Value']<=up_border) & (X_train['Value']>=low_border)].reset_index(drop=True)
    
    # prepare 'train_v' and 'valid_v' data frames - for validation 
    X_test = X_train 
    
    obs_in_test = 35
    X_train_v = X_train[:-obs_in_test].reset_index(drop=True)
    X_valid_v = X_train[-obs_in_test:].reset_index(drop=True)
    
    # prepare features 
        # for train
    X_train_ohe, X_test_ohe = feature_preprocessing(X_train, X_test, cat_cols, cat_type='ohe')
    X_train_meanenc, X_test_meanenc = feature_preprocessing(X_train, X_test, cat_cols, cat_type='mean_enc')
    
        # for validation 
    X_train_v_ohe, X_valid_v_ohe = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='ohe')
    X_train_v_meanenc, X_valid_v_meanenc = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='mean_enc')
    
        # group by mean
    y_hat_grby_mean = train_groupby(X_train_v, X_valid_v, window=999999, how='mean')
    y_hat_grby3_mean = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='mean')
    
        # group by median
    y_hat_grby_median = train_groupby(X_train_v, X_valid_v, window=999999, how='median')
    y_hat_grby3_median = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='median')
    
        # RandomForest 
    y_rf = train_rf(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe)
    y_rf_mean = train_rf(X_train_v_meanenc, X_train_v['Value'], X_valid_v_meanenc)
    y_rf_log = np.exp(train_rf(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe)) - 1
    y_rf_mean_log = np.exp(train_rf(X_train_v_meanenc, X_train_v['Value_log'], X_valid_v_meanenc)) - 1
    
        # LightGBM
   # y_lgb, lgb_opt = validate_lgb(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe, X_valid_v['Value'])
   # y_lgb_mean, lgb_mean_opt = validate_lgb(X_train_v_meanenc, X_train_v['Value'], X_valid_v_meanenc, X_valid_v['Value'])
   ## y_lgb_log, lgb_opt_log = validate_lgb(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe, X_valid_v['Value_log'])
   # y_lgb_log = np.exp(y_lgb_log) -1
   # y_lgb_mean_log, lgb_mean_opt_log = validate_lgb(X_train_v_meanenc, X_train_v['Value_log'], X_valid_v_meanenc, X_valid_v['Value_log'])
   # y_lgb_mean_log = np.exp(y_lgb_mean_log) -1
    
    # stack predictions and make predictions 
    X_valid_stack = combine_y_hats([y_hat_grby_mean, y_hat_grby3_mean, 
                                    y_hat_grby_median, y_hat_grby3_median, 
                                    y_rf, y_rf_mean, 
                                    y_rf_log, y_rf_mean_log, 
                                   # y_lgb, y_lgb_mean, 
                                   # y_lgb_log, y_lgb_mean_log
                                   ])
    #y_rf_hat = validate_stack(X_valid_stack, X_valid_v['Value'], rf_stack)
    y_lr_hat = cv_lr(X_valid_stack, X_valid_v['Value'], lr_stack)
 
    # calculate scores and pick top model 
    iid = X_valid_v.reset_index()['index'] 
    T = np.max(iid)
    index_mult = (3*T -2*iid +1) / 2 / T**2
    
    score_grby_mean = calc_score(y_hat_grby_mean, X_valid_v['Value'], index_mult)
    score_grby3_mean = calc_score(y_hat_grby3_mean, X_valid_v['Value'], index_mult)
    score_grby_median = calc_score(y_hat_grby_median, X_valid_v['Value'], index_mult)
    score_grby3_median = calc_score(y_hat_grby3_median, X_valid_v['Value'], index_mult)
    
    score_rf = calc_score(y_rf, X_valid_v['Value'], index_mult)
    score_rf_mean = calc_score(y_rf_mean, X_valid_v['Value'], index_mult)
    score_rf_log = calc_score(y_rf_log, X_valid_v['Value'], index_mult)
    score_rf_mean_log = calc_score(y_rf_mean_log, X_valid_v['Value'], index_mult)
    
    #score_lgb = calc_score(y_lgb, X_valid_v['Value'], index_mult)
   ## score_lgb_mean = calc_score(y_lgb_mean, X_valid_v['Value'], index_mult)
    #score_lgb_log = calc_score(y_lgb_log, X_valid_v['Value'], index_mult)
   # score_lgb_mean_log = calc_score(y_lgb_mean_log, X_valid_v['Value'], index_mult)

    score_lr_stack = calc_score(y_lr_hat, X_valid_v['Value'], index_mult)
    
    
    y_hats     = [y_hat_grby_mean, y_hat_grby3_mean, 
                  y_hat_grby_median, y_hat_grby3_median, 
                  y_rf, y_rf_mean, 
                  y_rf_log, y_rf_mean_log, 
                 # y_lgb, y_lgb_mean, 
                 # y_lgb_log, y_lgb_mean_log, 
                  y_lr_hat
                 ]
    
    all_scores = [score_grby_mean, score_grby3_mean, 
                  score_grby_median, score_grby3_median, 
                  score_rf, score_rf_mean, 
                  score_rf_log, score_rf_mean_log, 
                  #score_lgb, score_lgb_mean, 
                  #score_lgb_log, score_lgb_mean_log, 
                  score_lr_stack
                 ]
    
    best_score = np.min(all_scores)
    
    models_names = ['mean_all', 'mean_3', 
                    'median_all', 'median_3', 
                    'rf', 'rf_mean', 
                    'rf_log', 'rf_mean_log', 
                   # 'lgb', 'lgb_mean', 
                   # 'lgb_log', 'lgb_mean_log', 
                    'lr_stack'
                   ]
    # plot figures and seve to folder 
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
    ax.plot(y_hats[np.argmin(all_scores)])
    ax.plot(X_valid_v['Value'].reset_index(drop=True))
    fig.savefig('pics/'+str(i)+'.jpg')   # save the figure to file
    # plt.show()
    plt.close(fig)    # close the figure
    
    # calc R2 and save later 
    r2 = r2_score(y_hats[np.argmin(all_scores)], X_valid_v['Value'] ) 
    losses.append( r2 )
    
    
    
  
    print(i, 'Best model is', models_names[np.argmin(all_scores)], ' na value:', na_values_before, na_values_after)
    print('loss:', r2 )
    print(dict(zip(models_names, all_scores)))
    print('-------------------------------')


0 Best model is rf_log  na value: 0 0
loss: 0.29044351296387816
{'mean_all': 115873.92857142857, 'mean_3': 115873.92857142857, 'median_all': 114157.14285714286, 'median_3': 114157.14285714286, 'rf': 104140.33879066099, 'rf_mean': 118974.76113168437, 'rf_log': 102500.42108731047, 'rf_mean_log': 119055.63560765308, 'lr_stack': 105747.11416269254}
-------------------------------
2 Best model is rf_log  na value: 0 0
loss: -2.231490123871463
{'mean_all': 31451.836734693883, 'mean_3': 31451.836734693883, 'median_all': 25414.285714285714, 'median_3': 25414.285714285714, 'rf': 30328.381468946933, 'rf_mean': 36197.11129791062, 'rf_log': 22297.631092112075, 'rf_mean_log': 28371.50191721633, 'lr_stack': 33503.0749836989}
-------------------------------


/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


3 Best model is median_all  na value: 0 0
loss: -0.26324467418740016
{'mean_all': 114616.17346938775, 'mean_3': 114616.17346938775, 'median_all': 95085.71428571429, 'median_3': 95085.71428571429, 'rf': 100613.41156908429, 'rf_mean': 97372.97354482677, 'rf_log': 120946.98060961254, 'rf_mean_log': 104835.45919844457, 'lr_stack': 104786.06084893235}
-------------------------------
4 Best model is rf_log  na value: 0 0
loss: 0.42097832726297413
{'mean_all': 217057.9591836735, 'mean_3': 217057.9591836735, 'median_all': 192458.57142857142, 'median_3': 192458.57142857142, 'rf': 236961.2869808353, 'rf_mean': 313453.19043807214, 'rf_log': 151512.56476671985, 'rf_mean_log': 299193.55407696596, 'lr_stack': 160083.7869802524}
-------------------------------


/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


5 Best model is rf_mean_log  na value: 0 0
loss: 0.6132215037519412
{'mean_all': 205262.65306122447, 'mean_3': 205262.65306122447, 'median_all': 171951.42857142858, 'median_3': 171951.42857142858, 'rf': 187847.88211595733, 'rf_mean': 182663.58939641507, 'rf_log': 162077.298518407, 'rf_mean_log': 158589.5439786893, 'lr_stack': 181632.6454254011}
-------------------------------


...

/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


32 Best model is mean_all  na value: 0 0
loss: 0.8039085074601877
{'mean_all': 32538.26530612245, 'mean_3': 32538.26530612245, 'median_all': 32622.85714285714, 'median_3': 32622.85714285714, 'rf': 35596.73956014582, 'rf_mean': 32721.56743423171, 'rf_log': 45428.70757098086, 'rf_mean_log': 42111.89833919929, 'lr_stack': 39436.45262161209}
-------------------------------
33 Best model is rf_mean_log  na value: 0 0
loss: -4.425530025940663e+32
{'mean_all': 60170.40816326531, 'mean_3': 60170.40816326531, 'median_all': 35890.0, 'median_3': 35890.0, 'rf': 41779.28481165756, 'rf_mean': 29437.588403893547, 'rf_log': 27765.29970114125, 'rf_mean_log': 26827.12404777771, 'lr_stack': 46394.62314271897}
-------------------------------
34 Best model is median_all  na value: 0 0
loss: -8.788113876827106
{'mean_all': 15323.877551020409, 'mean_3': 15323.877551020409, 'median_all': 15184.285714285714, 'median_3': 15184.285714285714, 'rf': 17102.443445851106, 'rf_mean': 15849.858241006752, 'rf_log': 1602

/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


56 Best model is median_all  na value: 0 0
loss: -1.1426745443549637
{'mean_all': 250397.193877551, 'mean_3': 250397.193877551, 'median_all': 216272.85714285713, 'median_3': 216272.85714285713, 'rf': 286200.6319544509, 'rf_mean': 243611.89640740285, 'rf_log': 228957.02887511402, 'rf_mean_log': 228520.47087818378, 'lr_stack': 261596.4203311668}
-------------------------------
57 Best model is rf_log  na value: 0 0
loss: -0.4790873801262545
{'mean_all': 275087.85714285716, 'mean_3': 275087.85714285716, 'median_all': 272811.4285714286, 'median_3': 272811.4285714286, 'rf': 294480.3804220108, 'rf_mean': 339597.639000325, 'rf_log': 247792.3563914217, 'rf_mean_log': 342818.700579018, 'lr_stack': 296288.60417742113}
-------------------------------
58 Best model is mean_all  na value: 0 0
loss: -1.7285938158745373
{'mean_all': 63341.58163265306, 'mean_3': 63341.58163265306, 'median_all': 72625.71428571429, 'median_3': 72625.71428571429, 'rf': 79777.64338802014, 'rf_mean': 85418.10023662285, 'rf

KeyboardInterrupt: 